# Laboratorio #3 – Detección de Malware
### Jose Hernandez 20053
### Javier Mombiela 20067

#### Exploración de datos

In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('dataset.csv')

df.head()

,sha256,labels,0,1,2,3,4,5,6,7,...,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176
0,5c18291c481a192ed5003084dab2d8a117fd3736359218...,0,LdrUnloadDll,CoUninitialize,NtQueryKey,NtDuplicateObject,GetShortPathNameW,GetSystemInfo,IsDebuggerPresent,GetSystemWindowsDirectoryW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4683faf3da550ffb594cf5513c4cbb34f64df85f27fd1c...,0,NtOpenMutant,GetForegroundWindow,NtQueryKey,DrawTextExW,NtSetInformationFile,RegQueryValueExA,LdrGetProcedureAddress,CoUninitialize,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9a0aea1c7290031d7c3429d0e921f107282cc6eab854ee...,0,GetForegroundWindow,DrawTextExW,GetSystemInfo,IsDebuggerPresent,GetSystemWindowsDirectoryW,NtQueryValueKey,RegCloseKey,GetFileAttributesW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e0f3e4d5f50afd9c31e51dd9941c5a52d57c7c524f5d11...,0,NtQueryValueKey,LdrUnloadDll,GlobalMemoryStatus,WriteConsoleA,NtOpenKey,LdrGetProcedureAddress,NtTerminateProcess,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ec2b6d29992f13e74015ff0b129150b4afae15c593e4b7...,0,LdrUnloadDll,GetSystemTimeAsFileTime,NtOpenKey,WSAStartup,SetUnhandledExceptionFilter,NtTerminateProcess,NtClose,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
print("\nINFO:\n" , df.info())
print()
print(df['labels'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2570 entries, 0 to 2569
Columns: 177 entries, sha256 to Unnamed: 176
dtypes: int64(1), object(176)
memory usage: 3.5+ MB

INFO:
 None

labels
0    1285
1    1285
Name: count, dtype: int64


#### Preprocesamiento de datos e ingenieria de caracteristicas

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Concatena las columnas de las secuencias de llamadas a las API en una sola columna
df['sequence'] = df[df.columns[2:]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Crea un objeto TfidfVectorizer con n-gramas
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

# Ajusta el vectorizador a las secuencias y transforma las secuencias en vectores de n-gramas
X = vectorizer.fit_transform(df['sequence'])

# Obtiene las etiquetas
y = df['labels']

# Elimina las columnas originales de las secuencias de llamadas a las API
df = df.drop(df.columns[2:-1], axis=1)

# Muestra las primeras filas del conjunto de datos
df.head()

,sha256,labels,sequence
0,5c18291c481a192ed5003084dab2d8a117fd3736359218...,0,LdrUnloadDll CoUninitialize NtQueryKey NtDupli...
1,4683faf3da550ffb594cf5513c4cbb34f64df85f27fd1c...,0,NtOpenMutant GetForegroundWindow NtQueryKey Dr...
2,9a0aea1c7290031d7c3429d0e921f107282cc6eab854ee...,0,GetForegroundWindow DrawTextExW GetSystemInfo ...
3,e0f3e4d5f50afd9c31e51dd9941c5a52d57c7c524f5d11...,0,NtQueryValueKey LdrUnloadDll GlobalMemoryStatu...
4,ec2b6d29992f13e74015ff0b129150b4afae15c593e4b7...,0,LdrUnloadDll GetSystemTimeAsFileTime NtOpenKey...


#### Implementacion de modelos

In [55]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Arbol de Decision

In [56]:
from sklearn.tree import DecisionTreeClassifier

# Crea un clasificador de árbol de decisión
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Realiza una validación cruzada con K folds para k = 10 en el modelo de Árbol de Decisión
dt_cv_scores = cross_val_score(dt, X, y, cv=10)
print(f'Árbol de Decisión - Validación cruzada con K folds para k = 10: {dt_cv_scores.mean()}')

Árbol de Decisión - Validación cruzada con K folds para k = 10: 0.9252918287937744


SVM

In [57]:
from sklearn.svm import SVC

# Crea un clasificador de SVM
svm = SVC(random_state=42)
svm.fit(X_train, y_train)

# Realiza una validación cruzada con K folds para k = 10 en el modelo de SVM
svm_cv_scores = cross_val_score(svm, X, y, cv=10)
print(f'SVM - Validación cruzada con K folds para k = 10: {svm_cv_scores.mean()}')

SVM - Validación cruzada con K folds para k = 10: 0.9614785992217898


#### Calcular las metricas de cada modelo

In [59]:
dt_predictions = dt.predict(X_test)

# Calcula las métricas para el modelo de Árbol de Decisión
dt_accuracy = accuracy_score(y_test, dt_predictions)
dt_precision = precision_score(y_test, dt_predictions)
dt_recall = recall_score(y_test, dt_predictions)
dt_roc_auc = roc_auc_score(y_test, dt_predictions)

svm_predictions = svm.predict(X_test)

# Calcula las métricas para el modelo SVM
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)
svm_roc_auc = roc_auc_score(y_test, svm_predictions)

Comparacion de metricas

In [62]:
# Crea un DataFrame para las métricas
metrics = pd.DataFrame({
    'Modelo': ['Árbol de Decisión', 'SVM'],
    'Accuracy': [dt_accuracy, svm_accuracy],
    'Precision': [dt_precision, svm_precision],
    'Recall': [dt_recall, svm_recall],
    'ROC AUC': [dt_roc_auc, svm_roc_auc]
})

# Imprime el DataFrame
print(metrics)

              Modelo  Accuracy  Precision    Recall   ROC AUC
0  Árbol de Decisión  0.931258   0.940104  0.923274  0.931374
1                SVM  0.962387   0.994536  0.930946  0.962842


#### Métricas de rendimiento de cada modelo

**Árbol de Decisión**

El modelo de Árbol de Decisión obtuvo las siguientes métricas de rendimiento:

- Accuracy (Exactitud): 0.931258. Esto significa que el modelo hizo predicciones correctas (tanto positivas como negativas) en el 93.13% de las muestras.
- Precision (Precisión): 0.940104. Esto significa que el 94.01% de las muestras que el modelo predijo como positivas eran realmente positivas.
- Recall (Exhaustividad): 0.923274. Esto significa que el modelo identificó correctamente el 92.33% de todas las muestras positivas reales.
- ROC AUC (Área bajo la curva ROC): 0.931374. Esto significa que el modelo tiene una capacidad del 93.14% para distinguir entre clases positivas y negativas.

**SVM**

El modelo SVM obtuvo las siguientes métricas de rendimiento:

- Accuracy (Exactitud): 0.962387. Esto significa que el modelo hizo predicciones correctas (tanto positivas como negativas) en el 96.24% de las muestras.
- Precision (Precisión): 0.994536. Esto significa que el 99.45% de las muestras que el modelo predijo como positivas eran realmente positivas.
- Recall (Exhaustividad): 0.930946. Esto significa que el modelo identificó correctamente el 93.09% de todas las muestras positivas reales.
- ROC AUC (Área bajo la curva ROC): 0.962842. Esto significa que el modelo tiene una capacidad del 96.28% para distinguir entre clases positivas y negativas.


#### Comparación de los modelos
Al comparar las métricas de rendimiento de los dos modelos, el Árbol de Decisión y la Máquina de Vectores de Soporte (SVM), se observa que el modelo SVM supera al Árbol de Decisión en todas las métricas. En particular, el modelo SVM tiene una precisión significativamente más alta, lo que indica que es muy eficaz para predecir correctamente las muestras positivas. Sin embargo, ambos modelos muestran un buen rendimiento en la tarea de clasificación, lo que demuestra su utilidad para este tipo de tareas. A pesar de esto, los resultados obtenidos sugieren que el modelo SVM podría ser una mejor opción para esta tarea específica, dado su rendimiento superior en todas las métricas evaluadas. 